In [1]:
from dao.email import AVAILABLE_EMAIL_DAOS
from typing import List

/home/pawel/.virtualenvs/anti-gpt-checker/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/pawel/.virtualenvs/anti-gpt-checker/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [31]:
from dao.email import DAOEmailGenerated
from models.email import EmailGeneratedInDB
from tqdm import tqdm
from models.email import EmailGenerated, EmailGmailInDB
from services.email_generator import generate_email_based_on_og_email
from analysis.nlp_transformations import clean_text, separate_previous_conversation, remove_footers
from services.email_generator import _load_prompt_from_json, _request_to_openai_api, _ensure_no_placeholders
from models.email import EmailSpamAssassinInDB
dao_generated_emails = DAOEmailGenerated()
all_generated: List[EmailGeneratedInDB]= dao_generated_emails.find_all()
ids = [email.og_doc_id for email in all_generated]

In [32]:
from datetime import datetime

gmail_daos_info = [('gmail1', AVAILABLE_EMAIL_DAOS['gmail1']), ('gmail2', AVAILABLE_EMAIL_DAOS['gmail2']), ('gmail3', AVAILABLE_EMAIL_DAOS['gmail3'])]

max_per_dao = 500

for gmail_dao_name, gmail_dao in gmail_daos_info:
    emails: List[EmailGmailInDB] = gmail_dao.find_many_by_query({
        'email_labels': {'$regex': "Kategori"},
        "_id": {
            "$nin": ids
        },
        'detected_language': 'en'
    })
    print(gmail_dao_name)
    inserted_count = 0
    for email in tqdm(emails, total=len(emails), desc=f'Generating emails for {gmail_dao_name}', unit='emails', miniters=1):
        subject = email.subject
        if email.text_plain is None:
            continue

        if email.date > datetime(2022, 11, 30):
            continue

        _, email_text = separate_previous_conversation(email.text_plain)
        email_text = remove_footers(clean_text(email_text))
        if len(email_text) < 10:
            continue
        try:
            generated_text = generate_email_based_on_og_email(subject, email.text_plain, email.detected_language)
        except ValueError as e:
            continue
        except ConnectionError:
            continue
        placeholders_present = False
        if isinstance(generated_text, tuple):
            generated_text = generated_text[1]
            placeholders_present = True
        email_generated = EmailGenerated(
            og_db_name=gmail_dao_name,
            og_doc_id=email.id,
            subject=subject,
            text_plain=generated_text,
            language=email.detected_language,
            placeholders_present=placeholders_present,
            possible_advertisement=True
        )
        inserted_count += 1
        if inserted_count > max_per_dao:
            break
        dao_generated_emails.insert_one(email_generated)


gmail1


Generating emails for gmail1:  31%|███       | 1287/4197 [1:04:20<2:25:28,  3.00s/emails]


gmail2


Generating emails for gmail2:   4%|▎         | 32/854 [01:36<1:29:56,  6.57s/emails]

RESPONSE NOT FINISHED
Dear [Recipient's Name],

I hope this message finds you well. I am writing to inform you that there has been an unauthorized signup attempt for Google Cloud using your account. Google's security systems detected this activity and, as a precaution, have denied the signup. Your account is currently under review to ensure additional protection measures are in place.

To safeguard your account further, I recommend running a thorough check on your devices for any potential malware. Additionally, it is crucial to change your password immediately to prevent any further unauthorized access.

I want to reassure you that no unauthorized charges were made to your account during this incident. Google's priority is to maintain the security and integrity of all user accounts.

Thank you for your attention to this matter. If you have any concerns or require further assistance, please do not hesitate to reach out to our dedicated support team at support@googlecloud.com.

Warm reg

...

Generating emails for gmail3:  36%|███▋      | 512/1404 [18:53<35:39,  2.40s/emails]    

RESPONSE NOT FINISHED
Dear valued participants, Join the SunGod Momentum Creates Change challenge on Strava. Enhance your experience by joining clubs today. Visit our social media links for more details. Thank you. Best regards. John Smith Marketing Manager marketing@company.com 123 Marketing Street, Cityville ===FINISHED===


Generating emails for gmail3:  82%|████████▏ | 1150/1404 [1:18:00<21:06,  4.99s/emails]  

RESPONSE NOT FINISHED
Dear valued customers,

Don't miss out on our exclusive offer! Customize your PS4 or XB1 controller today and enjoy a €60 discount on orders over €159. Use the code "GAMER60" at checkout. Give your gaming experience a personal touch now!

Best regards,

John Doe===FINISHED===


Generating emails for gmail3:  83%|████████▎ | 1169/1404 [1:19:22<15:57,  4.07s/emails]


In [16]:
 other_dao_names = ['class_git', 'spam_assassin']
queries = [
    {'inner_classification': "personal"},
    {'is_spam': False}
]
for dao_name, query in zip(other_dao_names, queries):
    dao = AVAILABLE_EMAIL_DAOS[dao_name]
    query['_id'] = {
        "$nin": ids
    }
    emails: List[EmailGmailInDB] = dao.find_many_by_query(query)
    print(dao_name)
    for email in tqdm(emails, total=len(emails), desc=f'Generating emails for {dao_name}', unit='emails', miniters=1):
        subject = email.subject if len(email.subject) > 0 else "No subject"
        if email.text_plain is None:
            continue
        if len(email.text_plain) < 10:
            continue
        try:
            generated_text = generate_email_based_on_og_email(subject, email.text_plain, email.detected_language)
        except ValueError:
            continue
        except ConnectionError:
            continue
        placeholders_present = False
        if isinstance(generated_text, tuple):
            generated_text = generated_text[1]
            placeholders_present = True
        email_generated = EmailGenerated(
            og_db_name=dao_name,
            og_doc_id=email.id,
            subject=subject,
            text_plain=generated_text,
            language=email.detected_language,
            placeholders_present=placeholders_present
        )
        dao_generated_emails.insert_one(email_generated)

class_git


Generating emails for class_git: 0emails [00:00, ?emails/s]


spam_assassin


Generating emails for spam_assassin:   0%|          | 0/29 [00:00<?, ?emails/s]

not skipping


Generating emails for spam_assassin:   3%|▎         | 1/29 [00:01<00:29,  1.06s/emails]

not skipping


Generating emails for spam_assassin:   3%|▎         | 1/29 [00:11<05:31, 11.82s/emails]

KeyboardInterrupt



## FIXING EMAILS

In [8]:
fixed_ids = []

In [9]:
risky_generation = []

In [11]:
from services.email_generator import _load_prompt_from_json, _request_to_openai_api, _ensure_no_placeholders

misgenerated_emails = dao_generated_emails.find_many_by_query({
    'placeholders_present': True
})

for misgenerated_email in  tqdm(misgenerated_emails, total=len(misgenerated_emails), desc=f'Fixing emails', unit='email', miniters=1):
    try:
        messages = [
        {"role": "system",
         "content": _load_prompt_from_json("post_generation_fix", misgenerated_email.language)},
        {"role": "user",
         "content": misgenerated_email.text_plain}
        ]
        response_txt = _request_to_openai_api(messages)
    except ValueError:
        continue
    except ConnectionError:
        continue

    if _ensure_no_placeholders(response_txt):
        dao_generated_emails.update_one({"_id": misgenerated_email.id},
                                        {"$set": {
                                            'text_plain': response_txt,
                                            'placeholders_present': False
                                            }
                                        }
        )
        fixed_ids.append(misgenerated_email.id)
        risky_generation.append(misgenerated_email.id)
    else:
        continue

Fixing emails: 100%|██████████| 172/172 [23:51<00:00,  8.32s/email]


In [23]:
misgenerated_emails = dao_generated_emails.find_many_by_query({
    'placeholders_present': True
})
og_ids = [email.og_doc_id for email in misgenerated_emails]

In [24]:
dao_generated_emails.delete_many({
    'placeholders_present': True
})

3

In [25]:
len(og_ids)

3

In [70]:
misgenerated_emails = dao_generated_emails.find_many_by_query({
    'placeholders_present': True
})
og_ids2 = [email.og_doc_id for email in misgenerated_emails]
gmail1 = []
gmail2 = []
gmail3 = []
for email in misgenerated_emails:
    if email.og_db_name == 'gmail1':
        gmail1.append(email.og_doc_id)
    elif email.og_db_name == 'gmail2':
        gmail2.append(email.og_doc_id)
    elif email.og_db_name == 'gmail3':
        gmail3.append(email.og_doc_id)
def find_duplicates(input_list):
    counts = {}  # A dictionary to hold the count of each element
    duplicates = []  # A list to hold the duplicate elements

    # Count the occurrences of each element
    for item in input_list:
        counts[item] = counts.get(item, 0) + 1

    # Find elements that occur more than once
    for item, count in counts.items():
        if count > 4:
            duplicates.append(item)

    return duplicates

still_palceholders = find_duplicates(gmail3)

In [71]:
len(still_palceholders)

0

In [69]:
from models.email import EmailGmailInDB

dao = AVAILABLE_EMAIL_DAOS['gmail3']
query = {"_id":
    {
        "$in": still_palceholders
    }
}
emails: List[EmailGmailInDB] = dao.find_many_by_query(query)
for email in tqdm(emails, total=len(emails), desc=f'Generating emails', unit='emails', miniters=1):
    subject = email.subject if len(email.subject) > 0 else "No subject"
    if email.text_plain is None:
        continue
    if len(email.text_plain) < 10:
        continue
    try:
        generated_text = generate_email_based_on_og_email(subject, email.text_plain, email.detected_language)
    except ValueError:
        continue
    except ConnectionError:
        continue
    placeholders_present = False
    if isinstance(generated_text, tuple):
        generated_text = generated_text[1]
        placeholders_present = True
    email_generated = EmailGenerated(
        og_db_name='gmail3',
        og_doc_id=email.id,
        subject=subject,
        text_plain=generated_text,
        language=email.detected_language,
        placeholders_present=placeholders_present
    )
    dao_generated_emails.insert_one(email_generated)

Generating emails: 100%|██████████| 1/1 [00:03<00:00,  3.02s/emails]


In [72]:
dao_generated_emails.delete_many({
    'placeholders_present': True
})


132

## Detecting same content emails

In [26]:
gmail_daos_info = [('gmail1', AVAILABLE_EMAIL_DAOS['gmail1'])]#, ('gmail2', AVAILABLE_EMAIL_DAOS['gmail2']), ('gmail3', AVAILABLE_EMAIL_DAOS['gmail3'])]
added_number = 0

for gmail_dao_name, gmail_dao in gmail_daos_info:
    subject_list = []
    emails: List[EmailGmailInDB] = gmail_dao.find_many_by_query({
        "_id": {
            "$in": og_ids
        }
    })
    print(gmail_dao_name)
    for email in tqdm(emails, total=len(emails), desc=f'Generating emails for {gmail_dao_name}', unit='emails', miniters=1):
        subject = email.subject
        if subject in subject_list:
            continue
        subject_list.append(subject)
        if email.text_plain is None:
            continue
        _, email_text = separate_previous_conversation(email.text_plain)
        email_text = remove_footers(clean_text(email_text))
        if len(email_text) < 10:
            continue
        try:
            generated_text = generate_email_based_on_og_email(subject, email.text_plain, email.detected_language)
        except ValueError as e:
            print(e)
            continue
        except ConnectionError as e:
            print(e)
            continue
        placeholders_present = False
        if isinstance(generated_text, tuple):
            generated_text = generated_text[1]
            placeholders_present = True

        email_generated = EmailGenerated(
            og_db_name=gmail_dao_name,
            og_doc_id=email.id,
            subject=subject,
            text_plain=generated_text,
            language=email.detected_language,
            placeholders_present=placeholders_present,
            possible_advertisement=True
        )
        dao_generated_emails.insert_one(email_generated)
        added_number += 1
        if added_number > 1000:
            break


gmail1


Generating emails for gmail1: 100%|██████████| 3/3 [00:34<00:00, 11.40s/emails]
